In [54]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats

In [55]:
n = 21
x = np.linspace(0, 20, n)
w = np.array([-1.5, 1.0 / 9])
y = w[0] * x + w[1] * x ** 2 + stats.norm.rvs(size = n) * 2

In [59]:
def recursive_mean(n, x, xbar):
    return x if xbar == None else (xbar + (x - xbar) / (n + 1.0))

def recursive_var(n, x, xbar, xbar_next, cxx):
    if cxx == None:
        return 0
    else:
        return ((x ** 2 + n * cxx + n * xbar ** 2 - (n + 1) * xbar_next ** 2) / (n + 1.0))

def recursive_cov(n, x, xbar, xbar_next, y, ybar, ybar_next, cxy):
    if cxy == None:
        return 0
    else:
        return ((x * y + n * cxy + n * xbar * ybar - (n + 1) * xbar_next * ybar_next) / (n + 1.0))
    
def regress(n, xbar, ybar, cxx, cxy, x, y):    
    xbar_next = recursive_mean(n, x, xbar)
    ybar_next = recursive_mean(n, y, ybar)
    cxx = recursive_var(n, x, xbar, xbar_next, cxx)
    cxy = recursive_cov(n, x, xbar, xbar_next, y, ybar, ybar_next, cxy)
    w0 = 0
    w1 = 0
    return w0, w1, xbar, ybar, cxx, cxy 

In [60]:
xbar = None
ybar = None
cxx = None
cxy = None

for n in range(len(x)):
    w0, w1, xbar, ybar, cxx, cxy = regress(n, xbar, ybar, cxx, cxy, x[n], y[n])

plt.plot(x, y, 'ro')
plt.show()

TypeError: unsupported operand type(s) for ** or pow(): 'NoneType' and 'int'